In [1]:
import h3
import pandas as pd
import geopandas as gpd
import shapely
import pandas as pd
from shapely.geometry import MultiPolygon, Polygon, Point
import numpy as np
import contextily as ctx
import matplotlib.pyplot as plt
import geobr
import pyreadr
import matplotlib
%matplotlib inline

# Goal of this analysis
1. Load in social isolation (cell-phone mobility data), census demographic data, commuting survey data, and COVID-19 case data
2. Create exploratory visuals to understand distribution of regions within Sao Paulo
3. Spatial analysis to interpolate demographic, commuting, and COVID-19 data to the isolation hexagons for transferrable analysis

In [2]:
data_path = ''

### Load in Isolation Data
* Each isolation hexagon represents the proportion of individuals who live in a hexagon, who stay in a given day
    * The data is from a Brazilian mobile phone analytics company

In [3]:
iso= pd.read_csv(data_path + 'isolation_sp_h3_Mar-Sep2020.csv')

#get hexageon geometries from h3 package
iso['coords'] = [Polygon(h3.h3_to_geo_boundary(x, True)) for x in iso['h3']]
iso = gpd.GeoDataFrame(iso, geometry='coords', crs= {"init": "epsg:4326"})
iso.to_file('isolation.shp')
iso = gpd.read_file(data_path + 'isolation.shp')



In [5]:
#convert dates to datetime
iso['dt'] = pd.to_datetime(iso['dt'])

In [6]:
#get earliest and latest day
print(f"Earliest Date of Isolation Data: {iso['dt'].min()}")
print(f"Latest Date of Isolation Data: {iso['dt'].max()}")

Earliest Date of Isolation Data: 2020-03-01 00:00:00
Latest Date of Isolation Data: 2020-09-27 00:00:00


In [10]:
iso_plot = iso.drop_duplicates('h3')

In [1]:
iso_plot.reset_index(inplace=True)
iso_plot.to_csv('h3_ids.csv')

NameError: name 'iso_plot' is not defined

In [13]:
iso_plot = gpd.GeoDataFrame(iso_plot, geometry='coords', crs= {"init": "epsg:4326"})

/Users/shivyucel/anaconda3/envs/geo_env/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


### Sao Paulo Census Tract Shapefile

Get tract data from geobr package and save as shp

In [48]:

#get census tracts from geobr
tracts = geobr.read_census_tract(code_tract="SP", year=2010, simplified=False)
tracts.to_file(data_path + 'data-raw/cleaned/census_shp.shp')

#read in tract data
tracts = gpd.read_file(data_path + '/data-raw/cleaned/census_shp.shp')

iso_plot = iso_plot.to_crs(epsg=4674)

merged = gpd.sjoin(tracts, iso_plot, how='inner', op='intersects')

# Number of Census Tract Zones That Intersect with Mobility Index
len(merged['code_tract'].unique())

<ipython-input-48-7e00167f4285>:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  tracts.to_file(data_path + 'data-raw/cleaned/census_shp.shp')


# Interpolation

## Method:  Demographic Info (Race, Income, etc)

To compute demographic information for each hexagon from the census tract data, the first step is to determine the proportions of demographic characteristics in the census zone, by dividing by the total population (ex. income). Subsequently, the geographic proportion of each h3 hexagon covered by each census region is determined.

In [47]:
census_data = pd.read_csv(data_path + 'data-raw/census_tracts/census_tracts2010_sp.csv')

In [48]:
census_data.rename(columns={'pop_branca': 'pop_white', 'pop_preta': 'pop_black', 
                           'pop_amarela': 'pop_yellow', 'pop_parda': 'pop_brown', 
                           'pop_indigena': 'pop_indigenous'}, inplace=True)

## Income

In [ ]:
#create columns of income proportion relative to total population in each census tract
income_cols = []
for col in census_data.columns:
    if col[:6] == 'income':
        income_cols.append(col)
        

In [ ]:
income_cols += ['code_tract']
#set south american coordinate system
iso_plot = iso_plot.to_crs(epsg=4674)

overlay_iso_tract = gpd.overlay(iso_plot, tracts, how='intersection')

#get each intersection area between isolation hexagons and census tracts
overlay_iso_tract = overlay_iso_tract.merge(iso_plot[['h3', 'geometry']], on='h3')

#get proportion of isolation hexagon in each census tract-iso hexagon intersection area
overlay_iso_tract['overlap_proportion'] = [x.area/y.area for x, y in zip(overlay_iso_tract['geometry_x'], overlay_iso_tract['geometry_y'])]

#make code_tract as int for comparability
overlay_iso_tract.code_tract = overlay_iso_tract.code_tract.astype(int)

In [ ]:
#merge intersection areas with the proportions of demographic information from each census group
overlay_w_propor = overlay_iso_tract.merge(census_data[income_cols], how='left', on='code_tract' )

In [ ]:
#create weighted proportions for each h3 area's demographic info based on proportion of total area in given census tract
for col in income_cols:
    overlay_w_propor[f'weighted_{col}'] = overlay_w_propor[col] * overlay_w_propor['overlap_proportion']

In [ ]:
#create columns of demographic proportion relative to total population in each census tract
income_cols = []
for col in overlay_w_propor.columns:
    if col[:8] == 'weighted':
        income_cols.append(col)
        

In [ ]:
income_cols= income_cols[:-1]

In [ ]:
income_cols

['weighted_income_avg_head',
 'weighted_income_total',
 'weighted_income_0',
 'weighted_income_1',
 'weighted_income_2',
 'weighted_code_tract']

In [ ]:
overlay_w_propor = overlay_w_propor.iloc[:, :-1]

In [ ]:
h3_income = overlay_w_propor.groupby('h3')[income_cols].sum()

In [ ]:
h3_income

,weighted_income_avg_head,weighted_income_total,weighted_income_0,weighted_income_1,weighted_income_2
h3,,,,,
88a8100001fffff,1143.143445,219045.011075,7.212334,51.759904,148.251059
88a8100003fffff,928.374062,266227.034608,5.355643,76.529913,189.355202
88a8100005fffff,1004.591452,387886.510746,11.044067,94.836875,326.202259
88a8100007fffff,1160.208616,423527.487669,8.900752,90.437966,249.151094
88a8100009fffff,2069.354689,702143.748680,3.386166,68.824963,193.317424
...,...,...,...,...,...
88a812b4d7fffff,1115.374845,446098.731205,13.022348,117.310452,317.582862
88a812b4d9fffff,1486.043800,210526.126063,9.380995,45.194918,84.349785
88a812b4ddfffff,1513.275799,539765.072796,12.047683,89.093817,274.790064


In [ ]:
h3_income.to_csv(data_path + 'estimates/h3_income.csv')

In [ ]:
import h3
import pandas as pd
import geopandas as gpd
import shapely
import pandas as pd
from shapely.geometry import MultiPolygon, Polygon, Point
import numpy as np
import contextily as ctx
import matplotlib.pyplot as plt
import geobr
import pyreadr
import matplotlib
%matplotlib inline

### Get Commuting Flow Data

In [ ]:
commute = pd.read_csv('Data/data-raw/travel_survey/travel_matrix_spo2017.csv')
labels= pd.read_excel('Data/zone_labels.xlsx', skiprows=3, header = 1)

In [ ]:
#clean and merge dataframes
labels = labels[2:]
labels['Zona'] = labels[labels['Zona'] != 'nan']
labels['Zona'] = labels[labels['Zona'] != 'N°']
labels['Zona'] = labels[labels['Zona'] != 'Zona']
labels['Zona'] = labels['Zona'].astype(float)
labels.rename(columns={'Unnamed: 1': 'zona_nome', 'Unnamed: 3': 'municipio_nome', 'Unnamed: 5': 'distrito_nome'}, inplace=True)
labels.drop('Unnamed: 7', axis=1, inplace=True)
#commute = commute.merge(labels, left_on='ZONA_O', right_on='Zona')


# remove missing values
commute = commute[~commute['ZONA_O'].isnull()]
commute = commute[~commute['ZONA_D'].isnull()]
commute = commute[commute['ZONA_O'] != 'nan']
commute = commute[commute['ZONA_D'] != 'nan']
commute = commute[~commute['mean_time'].isnull()]

In [ ]:
commute_orig = commute_orig[['ZONA_O', 'pop_orig']]

In [ ]:
commute_orig.dropna(inplace=True)

In [ ]:
commute_orig['ZONA_O'] = commute_orig['ZONA_O'].astype(int)

In [ ]:
commute_orig.drop_duplicates('ZONA_O', inplace=True)

### Estimate 2020 Population for Commuting Zones and Estimate H3 Population Based on Degree of Overlap

#### Rate of Population Growth in Sao Paulo

In [ ]:
commute = pd.read_csv('Data/data-raw/travel_survey/travel_matrix_spo2017.csv')

In [ ]:
pop_SP_2010 = 11253503
pop_SP_2019 = 12252023

In [ ]:
delta_t = 9
yearly_rate = (pop_SP_2019/pop_SP_2010)**(1.0/delta_t)

print('Yearly population growth rate in SP:\n{:.6f} or {:2.4f}%'.format(yearly_rate, (yearly_rate-1)*100 ))

Yearly population growth rate in SP:
1.009490 or 0.9490%


In [ ]:
pop_SP_2020 = round(pop_SP_2010*(yearly_rate**10))

print('Estimated total SP population in 2020:',pop_SP_2020)

Estimated total SP population in 2020: 12368301


In [ ]:
commute_orig['2020_pop'] = commute_orig['pop_orig'] * yearly_rate**3

### Interpolate 2020-Scaled Commuting Region Populations

In [ ]:
import fiona
shape = fiona.open("Data/data-raw/travel_survey/Zonas_2017_region.shp")
# Build the GeoDataFrame from Fiona Collection
commute_zones = gpd.GeoDataFrame.from_features([feature for feature in shape], crs = 29193)
# Get the order of the fields in the Fiona Collection; add geometry to the end
columns = list(shape.meta["schema"]["properties"]) + ["geometry"]
# Re-order columns in the correct order
commute_zones = commute_zones[columns]

In [ ]:
commute_zones.to_crs(crs=4674, inplace=True)
iso_plot.to_crs(crs=4674, inplace=True)

In [ ]:
#Get area of commuting zones
commute_zones['zona_area'] = [x.area for x in commute_zones['geometry']]

#get intersections of isolation zones and commuting zones
iso_commute = gpd.overlay(iso_plot, commute_zones )

#get area of overlapping regions
iso_commute['overlap_area'] = [x.area for x in iso_commute['geometry']]


iso_commute = iso_commute[['h3', 'NumeroZona', 'overlap_area', 'zona_area']]

iso_commute = iso_commute.merge(commute_orig, left_on='NumeroZona', right_on='ZONA_O')

#get proportion of total commuting area covered by a given hexagon
iso_commute['overlap_propor'] = iso_commute['overlap_area'] / iso_commute['zona_area']

iso_commute['2020_pop_h3'] = iso_commute['overlap_propor'] * iso_commute['2020_pop'] 

iso_commute = iso_commute[['h3', '2020_pop_h3']]

In [ ]:
iso_commute.groupby('h3').sum().to_csv('Data/estimates/h3_pop.csv')

### Interpolate Commuting Flow Information to Social Isolation Hexagon Level

In [15]:
commute = pd.read_csv(data_path + '/data-raw/travel_survey/travel_matrix_spo2017.csv')
labels= pd.read_excel(data_path + '/zone_labels.xlsx', skiprows=3, header = 1)

In [16]:
#clean and merge dataframes
labels = labels[2:]
labels['Zona'] = labels[labels['Zona'] != 'nan']
labels['Zona'] = labels[labels['Zona'] != 'N°']
labels['Zona'] = labels[labels['Zona'] != 'Zona']
labels['Zona'] = labels['Zona'].astype(float)
labels.rename(columns={'Unnamed: 1': 'zona_nome', 'Unnamed: 3': 'municipio_nome', 'Unnamed: 5': 'distrito_nome'}, inplace=True)
labels.drop('Unnamed: 7', axis=1, inplace=True)
#commute = commute.merge(labels, left_on='ZONA_O', right_on='Zona')


# remove missing values
commute = commute[~commute['ZONA_O'].isnull()]
commute = commute[~commute['ZONA_D'].isnull()]
commute = commute[commute['ZONA_O'] != 'nan']
commute = commute[commute['ZONA_D'] != 'nan']
commute = commute[~commute['mean_time'].isnull()]

In [17]:
import fiona
shape = fiona.open(data_path + "/data-raw/travel_survey/Zonas_2017_region.shp")

In [19]:
# Build the GeoDataFrame from Fiona Collection
commute_zones = gpd.GeoDataFrame.from_features([feature for feature in shape], crs = 29193)
# Get the order of the fields in the Fiona Collection; add geometry to the end
columns = list(shape.meta["schema"]["properties"]) + ["geometry"]
# Re-order columns in the correct order
commute_zones = commute_zones[columns]

In [314]:
commute_zones.to_crs(crs=4674, inplace=True)
iso_plot.to_crs(crs=4674, inplace=True)

In [315]:
commute_zones.to_file('Data/data-raw/travel_survey/commute_shapes.shp')

commute_zones = gpd.read_file('Data/data-raw/travel_survey/Zonas_2017_region.shp')

#Get area of commuting zones
commute_zones['zona_area'] = [x.area for x in commute_zones['geometry']]

#get intersections of isolation zones and commuting zones
iso_commute = gpd.overlay(iso_plot, commute_zones )

#get area of overlapping regions
iso_commute['overlap_area'] = [x.area for x in iso_commute['geometry']]


iso_commute = iso_commute[['h3', 'FID', 'overlap_area', 'zona_area']]

#get proportion of total commuting area covered by a given hexagon
iso_commute['propor_overlap'] = iso_commute['overlap_area'] / iso_commute['zona_area']

#create dataframe to hold h3-commuting zone proportions
overlap_extent_matrix = pd.DataFrame(index=iso_plot['h3'], columns=commute_zones['NumeroZona'])

#fill in dataframe with proportions
for index, row in iso_commute.iterrows():
    overlap_extent_matrix.loc[row['h3'], row['FID']] = row['propor_overlap']

In [378]:
overlap_extent_matrix.replace(to_replace=np.nan, value=0, inplace=True)
overlap_extent_matrix = overlap_extent_matrix.transpose()
overlap_extent_matrix.to_csv('Data/data-raw/travel_survey/overlap_extent.csv')

In [669]:
#get commuting origins as strings
commute['ZONA_O'] = commute['ZONA_O'].astype(str)

#get overlap extent index as string
overlap_extent_matrix.index = overlap_extent_matrix.index.astype(str)

In [671]:
#create column to hold original commuting origin zone
commute['ZONA_O_orig'] = commute['ZONA_O']

In [675]:
overlap_extent_matrix.set_index('NumeroZona', inplace=True)

In [677]:
#replace origin commuting zone with all h3 zones which cover some proportion of the origin commuting zone
for zone in commute['ZONA_O'].unique():
    val = [col for col in overlap_extent_matrix.columns if overlap_extent_matrix.loc[zone, col] != 0]
    commute['ZONA_O'].replace(to_replace=zone, value= str(val), inplace=True)    

In [678]:
commute['ZONA_O'] = [ast.literal_eval(x) for x in commute['ZONA_O']]
commute = commute.explode('ZONA_O')

In [680]:
commute.reset_index(inplace=True, drop=True)

overlap_extent_matrix.reset_index(inplace=True)

In [683]:
pivoted = overlap_extent_matrix.melt(id_vars='NumeroZona')

#merge proportion of origin commuting zone covered by each overlapping h3
commute = commute.merge(pivoted, left_on=['ZONA_O_orig', 'ZONA_O'], right_on=['NumeroZona', 'h3'])

In [694]:
commute.rename(columns={'value': 'O_propor'}, inplace=True)

#multiply count of people from two commuting zones by the proportion of the origin commuting zone covered by the h3 
#gives estimate h3 --> commuting zone
commute['weighted_count'] = commute['O_propor'] * commute['count']

In [697]:
commute['ZONA_D'] = commute['ZONA_D'].astype(str)
overlap_extent_matrix.index = overlap_extent_matrix.index.astype(str)
commute['ZONA_D_orig'] = commute['ZONA_D']

overlap_extent_matrix.set_index('NumeroZona', inplace=True)

In [621]:
#get destination h3 zones which cover the commuting zone destinations
for zone in commute['ZONA_D'].unique():
    val = [col for col in overlap_extent_matrix.columns if overlap_extent_matrix.loc[zone, col] != 0]
    commute['ZONA_D'].replace(to_replace=zone, value= str(val), inplace=True)    

In [698]:
commute['ZONA_D'] = [ast.literal_eval(x) for x in commute['ZONA_D']]

In [703]:
commute = commute.explode('ZONA_D')

In [705]:
commute.reset_index(inplace=True, drop=True)

In [709]:
#get proportions of commuting destination zones covered by the intersecting h3 zones
commute = commute.merge(pivoted, left_on=['ZONA_D_orig', 'ZONA_D'], right_on=['NumeroZona', 'h3'])

In [710]:
#multiply 'h3' origin --> 'commuting zone' destination count by the proportion of destination commuting zone covered by h3
# get h3 --> h3 estimate
commute['final_count'] = commute['weighted_count'] * commute['value']

In [715]:
commute.to_csv('Data/data-raw/travel_survey/h3_commute.csv')

#### Create Clean OD Matrix of Hexagon Commuting

In [ ]:
h3_ids = pd.read_csv('/Users/shivyucel/Documents/SDS_2021.nosync/SDS_2020-2021/SDS_Thesis/Data/h3/h3_IDs.csv')

df = pd.read_csv(data_path + 'data-raw/travel_survey/h3_commute.csv')

df = df[['ZONA_O', 'ZONA_D', 'final_count']]

df = df.merge(h3_ids, left_on='ZONA_O', right_on='Unnamed: 0')

df = df.merge(h3_ids, left_on='ZONA_D', right_on='Unnamed: 0')

df = df.rename(columns={'0_x': 'SOURCE', '0_y': 'TARGET', 'final_count': 'FLUX'})[['SOURCE', 'TARGET', 'FLUX']]

df1 = pd.DataFrame(df.groupby(['SOURCE', 'TARGET'])['FLUX'].agg(sum))
df1.reset_index(inplace=True)

#df1.to_csv(data_path + 'h3/paper_clean_commute.csv')